# Getting Started with Space Language

Добро пожаловать в Space Language — систему многоуровневого анализа дискретных и непрерывных структур!

## Что такое Space Language?

Space Language объединяет **10 компонентов** в единый пайплайн:

- 🔄 **L0**: Система переписывания (дискретная динамика)
- 🤖 **L1**: Машина Тьюринга (вычисления)
- 📊 **L2**: Графы и SCC (структура)
- 🔍 **L3**: AC-автоматы (перекрытия)
- �� **L4**: Макро-система (семантика)
- 📝 **L5**: Генерация текста (AST)
- ✅ **L6**: Верификация (Z3, Isabelle)
- 📐 **L7**: Математические подъёмы (геометрия, меры, категории)

В этом туториале мы начнём с основ — системы переписывания строк.

## 1. Настройка окружения

Убедитесь, что путь к модулям Space Language добавлен в sys.path:

In [20]:
import sys
from pathlib import Path

# Добавляем src в путь
project_root = Path.cwd().parent
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"✓ Project root: {project_root}")
print(f"✓ Source path: {src_path}")

✓ Project root: /home/catman/Yandex.Disk/cuckoo/z/reals/libs/Experiments/Space/Lang
✓ Source path: /home/catman/Yandex.Disk/cuckoo/z/reals/libs/Experiments/Space/Lang/src


## 2. Основы: Строки и Правила

Система переписывания состоит из:
- **Строк** (`String`) — последовательности символов
- **Правил** (`Rule`) — преобразования вида `левая часть → правая часть`

Создадим простую систему для унарной арифметики:

In [21]:
from rewriting import String, Rule

# Создание строк
s1 = String.from_str("00")  # Два нуля
s2 = String.from_str("|")   # Штрих
s3 = String.from_str("0")   # Один ноль

print("Строки:")
print(f"  s1 = {s1}")
print(f"  s2 = {s2}")
print(f"  s3 = {s3}")
print()

# Создание правил
rule1 = Rule(String.from_str("00"), String.from_str("|"))   # 00 → |
rule2 = Rule(String.from_str("||"), String.from_str("0"))   # || → 0

print("Правила:")
print(f"  R1: {rule1.left} → {rule1.right}")
print(f"  R2: {rule2.left} → {rule2.right}")

Строки:
  s1 = 00
  s2 = |
  s3 = 0

Правила:
  R1: 00 → |
  R2: || → 0


## 3. Движок переписывания

`RewritingEngine` применяет правила и исследует пространство достижимых строк:

In [22]:
from rewriting import RewritingEngine

# Создаём движок с правилами
rules = [rule1, rule2]
engine = RewritingEngine(rules)

print(f"✓ Движок создан с {len(rules)} правилами")

✓ Движок создан с 2 правилами


## 4. Анализ достижимости

Исследуем, какие строки достижимы из начальной строки `"0000"`:

In [23]:
# Достижимость с полной историей преобразований
from collections import deque

initial = String.from_str("0000")  # Начальная строка
print(f"Начальная строка: {initial}")
print()

# BFS с сохранением истории: {строка -> (родитель, правило, позиция)}
history = {str(initial): None}  # Начальная строка без родителя
queue = deque([initial])
all_reachable = {initial}

max_depth = 10
depth = {str(initial): 0}

while queue:
    current = queue.popleft()
    current_depth = depth[str(current)]
    
    if current_depth >= max_depth:
        continue
    
    # Применяем все возможные правила
    for new_string, rule, pos in engine.all_applications(current):
        new_str = str(new_string)
        if new_str not in history:
            history[new_str] = (str(current), rule, pos)
            depth[new_str] = current_depth + 1
            all_reachable.add(new_string)
            queue.append(new_string)

# Функция восстановления пути
def get_path(target_str):
    """Восстанавливает путь от начальной строки до целевой."""
    path = []
    current = target_str
    while history[current] is not None:
        parent, rule, pos = history[current]
        path.append((current, rule, pos))
        current = parent
    path.append((str(initial), None, None))
    return list(reversed(path))

# Выводим все достижимые строки с полной историей
print(f"Найдено {len(all_reachable)} достижимых строк:")
print("=" * 60)

for i, s in enumerate(sorted(all_reachable, key=lambda x: (len(str(x)), str(x))), 1):
    s_str = str(s)
    path = get_path(s_str)
    
    print(f"\n{i}. Строка: '{s_str}' (глубина {depth[s_str]})")
    print("   История преобразований:")
    
    for j, (state, rule, pos) in enumerate(path):
        if rule is None:
            print(f"      [0] '{state}' (начало)")
        else:
            print(f"      [{j}] '{state}' ← применено {rule.left}→{rule.right} на позиции {pos}")

print("\n" + "=" * 60)
print(f"Итого: {len(all_reachable)} уникальных строк достижимы из '{initial}'")

Начальная строка: 0000

Найдено 6 достижимых строк:

1. Строка: '0' (глубина 3)
   История преобразований:
      [0] '0000' (начало)
      [1] '|00' ← применено 00→| на позиции 0
      [2] '||' ← применено 00→| на позиции 1
      [3] '0' ← применено ||→0 на позиции 0

2. Строка: '||' (глубина 2)
   История преобразований:
      [0] '0000' (начало)
      [1] '|00' ← применено 00→| на позиции 0
      [2] '||' ← применено 00→| на позиции 1

3. Строка: '00|' (глубина 1)
   История преобразований:
      [0] '0000' (начало)
      [1] '00|' ← применено 00→| на позиции 2

4. Строка: '0|0' (глубина 1)
   История преобразований:
      [0] '0000' (начало)
      [1] '0|0' ← применено 00→| на позиции 1

5. Строка: '|00' (глубина 1)
   История преобразований:
      [0] '0000' (начало)
      [1] '|00' ← применено 00→| на позиции 0

6. Строка: '0000' (глубина 0)
   История преобразований:
      [0] '0000' (начало)

Итого: 6 уникальных строк достижимы из '0000'


## 5. Последовательное применение правил

Посмотрим, как правила применяются шаг за шагом к строке `"0000"`:

In [27]:
# Последовательное применение правил
s = String.from_str("0000")
print(f"Начало: {s}")
print()

# Применяем правила пока возможно
step = 0
while True:
    apps = engine.all_applications(s)
    
    if not apps:
        print(f"\n✓ Достигнута строка '{s}', к которой нельзя применить ни одно правило")
        break
    
    # Показываем все возможные применения
    print(f"Шаг {step + 1}: строка '{s}'")
    print(f"   Возможные применения ({len(apps)}):")
    for new_s, rule, pos in apps:
        print(f"      • {rule.left}→{rule.right} на позиции {pos} → '{new_s}'")
    
    # Берём первое применение
    s, rule, pos = apps[0]
    print(f"   → Выбираем: {rule.left}→{rule.right} на позиции {pos}")
    print()
    step += 1
    
    if step > 10:  # Защита от бесконечного цикла
        print("... (прервано)")
        break

Начало: 0000

Шаг 1: строка '0000'
   Возможные применения (3):
      • 00→| на позиции 0 → '|00'
      • 00→| на позиции 1 → '0|0'
      • 00→| на позиции 2 → '00|'
   → Выбираем: 00→| на позиции 0

Шаг 2: строка '|00'
   Возможные применения (1):
      • 00→| на позиции 1 → '||'
   → Выбираем: 00→| на позиции 1

Шаг 3: строка '||'
   Возможные применения (1):
      • ||→0 на позиции 0 → '0'
   → Выбираем: ||→0 на позиции 0


✓ Достигнута строка '0', к которой нельзя применить ни одно правило


## 6. Нормальные формы

**Нормальная форма** — это строка, к которой нельзя применить ни одно правило из системы.

Найдём все нормальные формы среди достижимых строк:

In [28]:
# Нормальные формы — строки без возможных применений
normal_forms = []

print("Проверяем каждую достижимую строку:")
for s in sorted(all_reachable, key=lambda x: (len(str(x)), str(x))):
    apps = engine.all_applications(s)
    status = "нормальная форма ✓" if not apps else f"{len(apps)} применений"
    print(f"  '{s}': {status}")
    if not apps:
        normal_forms.append(s)

print()
print(f"Нормальные формы ({len(normal_forms)}):")
for nf in normal_forms:
    print(f"  • '{nf}' — к ней нельзя применить ни 00→|, ни ||→0")

Проверяем каждую достижимую строку:
  '0': нормальная форма ✓
  '||': 1 применений
  '00|': 1 применений
  '0|0': нормальная форма ✓
  '|00': 1 применений
  '0000': 3 применений

Нормальные формы (2):
  • '0' — к ней нельзя применить ни 00→|, ни ||→0
  • '0|0' — к ней нельзя применить ни 00→|, ни ||→0


## 7. Omega-предел (ω-предел)

**ω-предел** — это множество строк, в которых система "застревает" при бесконечном применении правил.

**Алгоритм:**
1. Начинаем с начальной строки
2. На каждом шаге применяем случайное правило (или все возможные)
3. Запоминаем все посещённые строки
4. Когда встречаем уже посещённую строку — нашли цикл
5. ω-предел = множество строк в этом цикле (аттрактор)

Покажем пошагово:

In [29]:
# Вычисление ω-предела вручную (показываем алгоритм)
import random

start = String.from_str("0000")
current = start
trajectory = [str(current)]  # История всех посещённых строк
visited_order = {str(current): 0}  # Когда впервые посетили строку

print(f"Начинаем с: '{current}'")
print()
print("Траектория (применяем случайные правила):")

for step in range(1, 21):
    apps = engine.all_applications(current)
    
    if not apps:
        # Нормальная форма — застряли навсегда
        print(f"  [{step}] '{current}' — нормальная форма (нет правил)")
        print()
        print(f"ω-предел = {{'{current}'}} — система застряла в одной точке")
        break
    
    # Случайный выбор правила
    new_string, rule, pos = random.choice(apps)
    new_str = str(new_string)
    
    print(f"  [{step}] '{current}' --({rule.left}→{rule.right} @{pos})--> '{new_str}'", end="")
    
    if new_str in visited_order:
        # Нашли цикл!
        cycle_start = visited_order[new_str]
        cycle = trajectory[cycle_start:]
        print(f"  ← ЦИКЛ! (возврат к шагу {cycle_start})")
        print()
        print(f"Обнаружен цикл длины {len(cycle)}:")
        print(f"  Цикл: {' → '.join(cycle)} → {new_str}")
        print()
        print(f"ω-предел = {set(cycle)}")
        print("(множество строк, которые бесконечно повторяются)")
        break
    else:
        print()
        visited_order[new_str] = step
        trajectory.append(new_str)
        current = new_string
else:
    print("... (цикл не найден за 20 шагов)")

Начинаем с: '0000'

Траектория (применяем случайные правила):
  [1] '0000' --(00→| @1)--> '0|0'
  [2] '0|0' — нормальная форма (нет правил)

ω-предел = {'0|0'} — система застряла в одной точке


In [31]:
# Все возможные ω-пределы системы
# (зависят от выбора правил на каждом шаге)

print("Анализ всех возможных исходов из '0000':")
print()

# Из анализа достижимости знаем все строки
# Нормальные формы — это точечные ω-пределы
print("Возможные ω-пределы:")
print()

for nf in normal_forms:
    print(f"  • {{'{nf}'}} — точечный аттрактор (нормальная форма)")

print()
print("В нашей системе нет циклов — все траектории заканчиваются")
print("в одной из нормальных форм: '0' или '0|0'")
print()
print("Какая именно — зависит от выбора правил на каждом шаге!")

Анализ всех возможных исходов из '0000':

Возможные ω-пределы:

  • {'0'} — точечный аттрактор (нормальная форма)
  • {'0|0'} — точечный аттрактор (нормальная форма)

В нашей системе нет циклов — все траектории заканчиваются
в одной из нормальных форм: '0' или '0|0'

Какая именно — зависит от выбора правил на каждом шаге!


## 📊 Резюме

В этом туториале вы узнали:

✅ Как создавать строки и правила  
✅ Как использовать `RewritingEngine`  
✅ Как находить достижимые строки с `bounded_reach`  
✅ Как определять нормальные формы  
✅ Как вычислять ω-предел системы  

## 🚀 Следующие шаги

Переходите к следующим туториалам:

- **02_graph_analysis.ipynb** — Построение графов G_L и SCC анализ
- **03_mathematical_lifts.ipynb** — Геометрические и мерные подъёмы
- **04_full_integration.ipynb** — Использование полного пайплайна